In [ ]:
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import pickle
from itertools import combinations

import pprint
import glob
from random import random, randrange
from random import choice
import logging
from collections import defaultdict
import sys

import subprocess
import shlex
from scipy.stats import pearsonr

import pandas as pd

import datetime
from datetime import datetime, timedelta
from dateutil.parser import parse
from collections import Counter
from scipy.signal import savgol_filter
import itertools
import time
import csv

import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


from sklearn.decomposition import LatentDirichletAllocation,NMF

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

from tqdm import trange, tqdm

In [ ]:
start_time = time.time()

In [ ]:
# Using the regular dictionary
f = open('pruned_retweet_dict','rb')
retweet_dict = pickle.load(f)
f.close()

PT = nx.read_gexf('PT-pruned.gexf')

In [ ]:
# clusters we care about
clusters = [13,48,6,49,104]
cluster_names = ['Democrats',"Republicans","Unorthodox","Public Health","Antivaxxers"]

In [ ]:
def author_to_cluster(author):
    return(PT.nodes[author]['louvain'])

In [ ]:
mininglist1 = ['vax', 'vaxxed', 'vaccine', 'vaccination', 'vaccinations', 
'vaxsafety', 'vax saftey', 'vaccineswork', 'vaccines work', 'vaccinesaftey',
'vaccine saftey', 'vaccines revealed', 'vaccinesrevealed', 
'novax', 'no vax', 'no-vax', 'antivax', 'anti-vax', 'anti vax', 'immunisation', 
'Vaccin', 'Vaccinaties', 'vaccinatiezorg', 'vaccine injury', 'vax injury',
'vaccinatieschade']

extralist = ['vaccines','coronavirus','corona','covid19','virus','vaccine','covid','covid-19',\
             'covidー19','#covidー19','vaccin','im','covid19','would',\
            'dont', 'year', 'going', 'months', 'first', 'new','covid_19','africansarenotlabrats',\
            'wwg1wga','sarscov2','covid2019','wwgwga']

remove_terms = np.array([])
remove_terms = np.append(remove_terms,mininglist1)
#remove_terms = np.append(remove_terms,mininglist2)
remove_terms = np.append(remove_terms,extralist)
remove_terms

In [ ]:
import re

# Def Remove emoji from string
def deEmojify(s):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',s)


# Def remove arabic
# According to Wikipedia, Arabic characters fall in the Unicode range 0600 - 06FF. 
def remove_foreign(s): 
    arabic_list = re.findall(r'[\u0600-\u06FF]+',s)
    for w in arabic_list:
        s = s.replace(w, " ")
        
    chinese_list = re.findall(r'[\u4e00-\u9FFF]+',s)
    for w in chinese_list:
        s = s.replace(w, " ")
        
    hindi_list = re.findall(r'[\u0900-\u097F]+',s)
    for w in hindi_list:
        s = s.replace(w, " ")
             
    korean_list = re.findall(r'[\u3040-\u30ff]+',s)
    for w in korean_list:
        s = s.replace(w, " ")
              
    japanese_list = re.findall(r'[\u3040-\u30ff]+',s)
    for w in japanese_list:
        s = s.replace(w, " ")
        
    return(s)

from string import digits
remove_digits = str.maketrans('', '', digits)

def light_preprocessor(s):
    s = s.strip().lower()
    s = s.replace('\n'," ")
    s = s.replace('-', " ")
    s = s.replace('.', "")
    # Remove Emojis
    s = deEmojify(s)
    # Remove Arabic
    s = remove_foreign(s)
    
    s = re.sub("\s+"," ",s)
    
    s = s.translate(remove_digits)
    
    s = [x for x in s if (x.isalnum() or x == " " or x=="#" or x=="@")]
    s = "".join(s)
     # Remove stopwords in english and french
    stop_words = stopwords.words('english') + stopwords.words('french')
    s = " ".join([x for x in s.split() if ((x not in stop_words) 
                                           and (x.find('http') == -1) 
                                           #and (x[0] != '#')
                                           #and (x[0] != '@')
                                          and (x != 'amp')
                                         and (x not in remove_terms))])
    s = s.replace('/',' ')
    
    return s

In [ ]:
print("Unfiltered version:")
for item in retweet_dict[1212147705507700736]['hashtags']:
    #item = item.lower()
    #if item not in remove_terms:
    print(item)

print("Filtered version:")
for item in retweet_dict[1212147705507700736]['hashtags']:
    item = light_preprocessor(item)
    print("'" + item + "'")


In [ ]:
def create_hashtag_corpus(date1,date2):
    date1 = datetime.strptime(date1,"%d/%m/%y")
    date2 = datetime.strptime(date2,"%d/%m/%y")
    corpus = dict()
    for c in clusters:
        corpus[str(c)]=""

    for t in tqdm(retweet_dict):
        a = retweet_dict[t]['author']
        try:
            c = author_to_cluster(a)
            if c in clusters:
                rt_date = datetime.strptime(retweet_dict[t]['date'],'%y-%m-%d-%H:%M:%S')
                if (date1 <= rt_date) and (rt_date <= date2):
                    hashtag_list = retweet_dict[t]['hashtags']
                    for hashtag in hashtag_list:
                        hashtag = light_preprocessor(hashtag)
                        text = " " + hashtag
                    corpus[str(c)] += text
        except KeyError:
            pass
        
    # remove double spaces
    for key in corpus:
        for i in range(4):
            corpus[key] = corpus[key].replace('  ', " ")
    return corpus

In [ ]:
# Testing
test = create_hashtag_corpus('26/12/19','28/12/19')
test['104']

In [ ]:
create_corpus = False

In [ ]:
# Create Corpus
if create_corpus:
    corpus = create_hashtag_corpus('26/12/19','27/05/20')
    f = open("hastag_corpus.pkl","wb")
    pickle.dump(corpus,f)
    f.close()

In [ ]:
f = open('hastag_corpus.pkl','rb')
corpus = pickle.load(f)
f.close()

In [ ]:
corpus['6']

In [ ]:
true_corpus = []
for cluster in clusters:
    true_corpus.append(corpus[str(cluster)])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.5,max_df=0.9,lowercase=True)
tfidf = vectorizer.fit_transform(true_corpus)
feature_names = np.array(vectorizer.get_feature_names())

# To get the cool hashtags:
# vectorizer = TfidfVectorizer(min_df=0.5,max_df=0.9,lowercase=True)


In [ ]:
# See second answer of:
# https://stackoverflow.com/questions/34232190/scikit-learn-tfidfvectorizer-how-to-get-top-n-terms-with-highest-tf-idf-score
def get_top_tf_idf_words(response, top_n=2):
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]]
  
print([get_top_tf_idf_words(response,15) for response in tfidf])